<img src="Images/nvidia_header.png" style="margin-left: -30px; width: 300px; float: left;">

## Exercise: Computing Variance

So far, you've learned how to compute mean variance and maximal difference.
In this exercise, you'll apply techniques of extending standard algorithms that we just covered.
This time, you'll be implementing an efficient variance algorithm.
Variance is computed on a sequence of data.
It measures how far the values in the sequence are spread from the mean:

$$
\frac{\sum\left(x_{i} - \overline{x} \right)^{2}}{N}
$$

As the equation above suggests, for each value in the sequence we have to compute the squared difference between this value and mean.
We then add all those squared differences together and divide the resulting sum by `N`. 

The next exercise consists of using a transform iterator to compute the squared differences.

Transform iterator API for your reference:

```c++
int constant = 2;
auto transform_it = thrust::make_transform_iterator(
    // iterator to the beginning of the input sequence
    vector.begin(), 
    // capture constant in the lambda by value with `[name]`
    [constant]__host__ __device__(float value_from_input_sequence) { 
      // transformation of each element
      return value_from_input_sequence * constant; 
    });
```

Use `thrust::reduce` to compute the sum of squared differences. 

In [ ]:
%%writefile Sources/variance.cu
#include "ach.h"

float variance(const thrust::universal_vector<float> &x, float mean) {
  // update the following line so that dereferencing `squared_differences`
  // returns `(xi - mean) * (xi - mean)`
  auto squared_differences = ...;

  return thrust::reduce(thrust::device, squared_differences,
                        squared_differences + x.size()) /
         x.size();
}

float mean(thrust::universal_vector<float> vec) {
  return thrust::reduce(thrust::device, vec.begin(), vec.end()) / vec.size();
}

int main() {
  float ambient_temp = 20;
  thrust::universal_vector<float> prev{42, 24, 50};
  thrust::universal_vector<float> next{0, 0, 0};

  std::printf("step  variance\n");
  for (int step = 0; step < 3; step++) {
    thrust::transform(thrust::device, prev.begin(), prev.end(), next.begin(),
                      [=] __host__ __device__(float temp) {
                        return temp + 0.5 * (ambient_temp - temp);
                      });
    std::printf("%d     %.2f\n", step, variance(next, mean(next)));
    next.swap(prev);
  }
}

In [ ]:
!nvcc --extended-lambda -o /tmp/a.out Sources/variance.cu # build executable
!/tmp/a.out # run executable

The output of your program should be:

| Step | Variance |
|------|----------|
| 0    | 29.56    |
| 1    | 7.39     |
| 2    | 1.85     |

If you’re unsure how to proceed, consider expanding this section for guidance. Use the hint only after giving the problem a genuine attempt.

<details>
  <summary>Hints</summary>
  
  - You can capture mean in a lambda function with `[mean]__host__ __device__ (...` 
  - You can transform the input sequence into squared differences with `thrust::transform_iterator`
  - You can create a transform iterator with `thrust::make_transform_iterator`
</details>

Open this section only after you’ve made a serious attempt at solving the problem. Once you’ve completed your solution, compare it with the reference provided here to evaluate your approach and identify any potential improvements.

<details>
  <summary>Solution</summary>

  Key points:
  - use `thrust::make_transform_iterator`
  - capture mean by value in lambda

  Solution:
  ```c++
  auto squared_differences = thrust::make_transform_iterator(
    x.begin(), [mean] __host__ __device__(float value) {
      return (value - mean) * (value - mean);
    });
  ```

  You can find full solution [here](Solutions/variance.cu).
</details>

---
Congratulations!  Now that you know how to extend standard algorithms, proceed to the [next section](../01.04-Vocabulary-Types/01.04.01-Vocabulary-Types.ipynb).

<img src="Images/nvidia_header.png" style="margin-left: -30px; width: 300px; float: left;">